In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

# Read the CSV and Perform Basic Data Cleaning

In [4]:
# https://help.lendingclub.com/hc/en-us/articles/215488038-What-do-the-different-Note-statuses-mean-

columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [5]:
# Load the data
file_path = Path('../Module-17-Challenge-Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [6]:
#pd.set_option('display.max_rows', None)

df.dtypes

loan_amnt                     float64
int_rate                      float64
installment                   float64
home_ownership                 object
annual_inc                    float64
                               ...   
total_bal_ex_mort             float64
total_bc_limit                float64
total_il_high_credit_limit    float64
hardship_flag                  object
debt_settlement_flag           object
Length: 86, dtype: object

In [7]:
df_encoded = pd.get_dummies(df,columns=["debt_settlement_flag","hardship_flag","application_type","next_pymnt_d","initial_list_status","pymnt_plan","issue_d","verification_status","home_ownership"])

In [8]:
#df_encoded


# Split the Data into Training and Testing

In [9]:
# Create our features
X = df_encoded.copy()
X = X.drop("loan_status", axis=1)
X.head()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,issue_d_Feb-2019,issue_d_Jan-2019,issue_d_Mar-2019,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,home_ownership_ANY,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT
0,10500.0,0.1719,375.35,66000.0,27.24,0.0,0.0,8.0,0.0,1609.0,...,0,0,1,0,1,0,0,0,0,1
1,25000.0,0.2000,929.09,105000.0,20.23,0.0,0.0,17.0,1.0,18368.0,...,0,0,1,0,0,1,0,1,0,0
2,20000.0,0.2000,529.88,56000.0,24.26,0.0,0.0,8.0,0.0,13247.0,...,0,0,1,0,0,1,0,1,0,0
3,10000.0,0.1640,353.55,92000.0,31.44,0.0,1.0,10.0,1.0,17996.0,...,0,0,1,0,0,1,0,0,0,1
4,22000.0,0.1474,520.39,52000.0,18.76,0.0,1.0,14.0,0.0,9091.0,...,0,0,1,1,0,0,0,1,0,0


In [10]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,issue_d_Feb-2019,issue_d_Jan-2019,issue_d_Mar-2019,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,home_ownership_ANY,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,0.371696,0.451066,0.177238,0.478007,0.373992,0.148001,0.009285,0.526309,0.106747,0.357659
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.483261,0.497603,0.381873,0.499520,0.483865,0.355104,0.095914,0.499311,0.308793,0.479314
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [11]:
# Create our target
y = df["loan_status"]

In [12]:
# Check the balance of our target values
y.value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

# Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble AdaBoost classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Print the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [14]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [15]:
# Splitting into Train and Test sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [16]:
#Counter(y_resampled)

In [17]:
# Resample the training data with the BalancedRandomForestClassifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=1) 

In [18]:
# Fitting the model
rf_model = rf_model.fit(X_train, y_train)

In [19]:
# Calculated the balanced accuracy score
y_pred = rf_model.predict(X_test)
acc_score =balanced_accuracy_score(y_test, y_pred)

In [20]:
print(f"Accuracy Score : {acc_score}")

Accuracy Score : 0.673150395020793


In [21]:
# Create a DataFrame from the confusion matrix.
cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

In [22]:
# Display the confusion matrix
print("Confusion Matrix")
display(cm_df)

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,35,66
Actual 1,4,17100


In [23]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print("Classification Report")
print(classification_report_imbalanced(y_test, y_pred))

Classification Report
                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.90      0.35      1.00      0.50      0.59      0.32       101
   low_risk       1.00      1.00      0.35      1.00      0.59      0.37     17104

avg / total       1.00      1.00      0.35      1.00      0.59      0.37     17205



In [24]:
# Calculate feature importance in the Random Forest model.
importances = rf_model.feature_importances_
importances

array([0.0159536 , 0.01113354, 0.02124031, 0.01255832, 0.01649291,
       0.00360554, 0.00555766, 0.00912512, 0.00255687, 0.01556922,
       0.01259458, 0.02158558, 0.01794206, 0.0610839 , 0.07908311,
       0.04811441, 0.06436359, 0.01380184, 0.        , 0.        ,
       0.06968104, 0.00168367, 0.        , 0.        , 0.00544187,
       0.01378285, 0.0060702 , 0.00634463, 0.00447453, 0.00699579,
       0.01180891, 0.01332987, 0.01184924, 0.00570435, 0.00690242,
       0.01676142, 0.01030777, 0.01365263, 0.00680063, 0.00870705,
       0.00909673, 0.00843629, 0.01414653, 0.01399067, 0.01396538,
       0.00050585, 0.        , 0.01493906, 0.01482792, 0.00903952,
       0.00891546, 0.00697957, 0.00890023, 0.01081607, 0.00397837,
       0.00681674, 0.00821767, 0.00776202, 0.0091682 , 0.01041863,
       0.00712209, 0.01103298, 0.00829272, 0.00822237, 0.        ,
       0.        , 0.00179289, 0.00724528, 0.00857719, 0.00648185,
       0.0032269 , 0.        , 0.01221391, 0.01390873, 0.01276

In [25]:
# List the features sorted in descending order by feature importance
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.07908310829750982, 'total_pymnt_inv'),
 (0.06968103901837423, 'last_pymnt_amnt'),
 (0.06436359075074595, 'total_rec_int'),
 (0.0610838995065997, 'total_pymnt'),
 (0.048114414599983314, 'total_rec_prncp'),
 (0.02158558042565099, 'out_prncp'),
 (0.021240305526111097, 'installment'),
 (0.017942063922377554, 'out_prncp_inv'),
 (0.01676142417873521, 'max_bal_bc'),
 (0.016492911028426697, 'dti'),
 (0.015953601192934656, 'loan_amnt'),
 (0.015569220611275754, 'revol_bal'),
 (0.014939058044943797, 'mo_sin_old_il_acct'),
 (0.01482791904701174, 'mo_sin_old_rev_tl_op'),
 (0.014146527109290125, 'avg_cur_bal'),
 (0.013990671245115353, 'bc_open_to_buy'),
 (0.013965377730542154, 'bc_util'),
 (0.013908734413005016, 'total_bal_ex_mort'),
 (0.01380184133098309, 'total_rec_late_fee'),
 (0.01378284986563163, 'tot_cur_bal'),
 (0.013652634245759295, 'total_rev_hi_lim'),
 (0.013545788272864262, 'total_il_high_credit_limit'),
 (0.013329871217758584, 'total_bal_il'),
 (0.012764088430375777, 'total_bc_limit'

### Easy Ensemble AdaBoost Classifier

In [ ]:
from imblearn.ensemble import EasyEnsembleClassifier 
# Train the EasyEnsembleClassifier
#X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    random_state=1)

In [ ]:
#eec = EasyEnsembleClassifier(random_state=1)
#eec.fit(X_train, y_train) 

In [ ]:
# Display the confusion matrix
#y_pred = eec.predict(X_test)
#print(confusion_matrix(y_test, y_pred))

In [ ]:
# Calculated the balanced accuracy score
#from sklearn.metrics import balanced_accuracy_score
#balanced_accuracy_score(y_test, y_pred)

In [ ]:
# Print the imbalanced classification report
#from imblearn.metrics import classification_report_imbalanced
#print("Classification Report")
#print(classification_report_imbalanced(y_test, y_pred))

In [ ]:
#from sklearn.ensemble import AdaBoostClassifier
#ad_model = AdaBoostClassifier(n_estimators=100, random_state=1)
#pred = ad.fit(x_train,y_train).predict(x_test)

In [ ]:
# Calculated the balanced accuracy score
#accuracy_score(y_test,pred)

In [ ]:
# Print the imbalanced classification report
#from imblearn.metrics import classification_report_imbalanced
#print("Classification Report")
#print(classification_report_imbalanced(y_test, y_pred))

In [ ]:
# Display the confusion matrix
#y_pred = ad_model.predict(X_test)
#print(confusion_matrix(y_test, y_pred))

In [26]:
from sklearn.ensemble import AdaBoostClassifier
# Train the EasyEnsembleClassifier
X_train, X_test, y_train, y_test = train_test_split(X, y,random_state=1)

In [27]:
#from sklearn.tree import DecisionTreeClassifier
#model = DecisionTreeClassifier(criterion='entropy', max_depth=1)

In [28]:
ad_model = AdaBoostClassifier(n_estimators=100, learning_rate=1)

In [29]:
boost_model = ad_model.fit(X_train,y_train)

In [30]:
y_pred = boost_model.predict(X_test)

In [31]:
from sklearn import metrics 
prediction = metrics.accuracy_score(y_test,y_pred)

In [32]:
print('The accuracy is: ', prediction * 100, '%')

The accuracy is:  99.60476605637896 %


In [33]:
# Create a DataFrame from the confusion matrix.
ada_cm = confusion_matrix(y_test, y_pred)
ada_cm_df = pd.DataFrame(
    ada_cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

In [34]:
# Display the confusion matrix
print("Confusion Matrix")
display(ada_cm_df)

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,38,63
Actual 1,5,17099


In [35]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print("Classification Report")
print(classification_report_imbalanced(y_test, y_pred))

Classification Report
                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.88      0.38      1.00      0.53      0.61      0.35       101
   low_risk       1.00      1.00      0.38      1.00      0.61      0.40     17104

avg / total       1.00      1.00      0.38      1.00      0.61      0.40     17205

